In [ ]:
!pip install sacremoses transformers datasets torch huggingface-hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
'''
In below cell paste the command/code provided by model in hugging face that you want to fine tune.
'''

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="microsoft/biogpt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/696k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
'''
In below two cells paste the command/code provided by dataset in hugging face that you want model to be fine tuned on.
OR
You can paste the path of dataset that you have downloaded.
'''

In [ ]:
# Load dataset
from datasets import load_dataset

ds = load_dataset("rag-datasets/rag-mini-bioasq", "question-answer-passages")

README.md:   0%|          | 0.00/813 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4719 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset

ds = load_dataset("rag-datasets/rag-mini-bioasq", "text-corpus")

part.0.parquet:   0%|          | 0.00/24.5M [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/40221 [00:00<?, ? examples/s]

In [ ]:
'''
In below cell is complete code of model fine tuning. Replace the model and dataset path. Also first create wandb id also as it will ask for it.
NOTE: You might have to adjust the parameters according to your needs as the model accuracy may be low.
'''

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch

# Load the dataset
ds = load_dataset("rag-datasets/rag-mini-bioasq", "question-answer-passages")

# Print available splits to inspect the correct ones
print(ds.keys())  # This will show all available dataset splits

# Load the BioGPT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
model = AutoModelForCausalLM.from_pretrained("microsoft/biogpt")

# Preprocess the dataset (tokenization)
def preprocess_function(examples):
    inputs = tokenizer(examples['question'], truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(examples['answer'], truncation=True, padding="max_length", max_length=512)
    inputs['labels'] = labels['input_ids']
    return inputs

# Access the dataset splits based on the output from print(ds.keys())
# Using the 'test' split as you mentioned
test_dataset = ds['test'].map(preprocess_function, batched=True)

# Since there is no separate validation split, you can use the same test dataset for evaluation
eval_dataset = test_dataset

training_args = TrainingArguments(
    output_dir="./results",            # Where to store the trained model
    evaluation_strategy="steps",       # Evaluate every few steps
    save_strategy="steps",            # Save model every few steps (matching evaluation strategy)
    eval_steps=500,                    # Evaluate every 500 steps
    save_steps=500,                    # Save model every 500 steps
    learning_rate=2e-5,                # Learning rate for fine-tuning
    per_device_train_batch_size=4,     # Batch size for training
    per_device_eval_batch_size=4,      # Batch size for evaluation
    num_train_epochs=3,                # Number of training epochs
    weight_decay=0.01,                 # Weight decay for regularization
    logging_dir="./logs",              # Directory for logs
    logging_steps=10,                  # Log every 10 steps
    save_total_limit=2,                # Keep only 2 saved models to prevent storage issues
    load_best_model_at_end=True,       # Load the best model after training
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=test_dataset,      # Using 'test' for training (as you mentioned)
    eval_dataset=eval_dataset,       # Same for evaluation
)

# Start training
trainer.train()

# Save the model after training
model.save_pretrained("./fine_tuned_biogpt")
tokenizer.save_pretrained("./fine_tuned_biogpt")

dict_keys(['test'])


Map:   0%|          | 0/4719 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,1.090000,0.922361
1000,0.888900,0.897004
1500,0.882100,0.865051
2000,0.961000,0.842366
2500,0.724400,0.827617
3000,1.140400,0.816826
3500,0.857700,0.810844


There were missing keys in the checkpoint model loaded: ['output_projection.weight'].


('./fine_tuned_biogpt/tokenizer_config.json',
 './fine_tuned_biogpt/special_tokens_map.json',
 './fine_tuned_biogpt/vocab.json',
 './fine_tuned_biogpt/merges.txt',
 './fine_tuned_biogpt/added_tokens.json')

In [ ]:
'''================================================================Fine Tunning Completed============================================================='''

In [1]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
'''
First create model in huggingface before running the code and paste the model name in place of "harishussain12/SkyeMed"
'''

In [2]:
from huggingface_hub import hf_hub_download
from huggingface_hub import HfApi

api = HfApi()

api.upload_folder(
    folder_path = "./fine_tuned_healthcare_gpt", #Folder/file name
    repo_id = "harishussain12/SkyeMed",  # Model created name in hugging face
    repo_type = "model",
)

model.safetensors: 100% 498M/498M [00:30<00:00, 23.2MB/s]
'CommitInfo(commit_url='https://huggingface.co/harishussain12/SkyeMed/commit/0693539360a43f456bc4cfe86bdeb193e44981b9', commit_message='Upload folder using '
huggingface_hub', commit_description='', oid='0693539360a43f456bc4cfe86bdeb193e44981b9', pr_url=None, 
repo_url=RepoUrl('https://huggingface.co/harishussain12/SkyeMed', endpoint='https://huggingface.co', repo_type='model', repo_id='harishussain12/SkyeMed'),
pr_revision=None, pr_num=None)
